<a href="https://colab.research.google.com/github/Liza-IITP/MachineLearning/blob/main/Ridge_GD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from sklearn.datasets import load_diabetes
from sklearn.metrics import r2_score
import numpy as np

In [39]:
X,y = load_diabetes(return_X_y=True)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [40]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(max_iter=1000,eta0=0.1,learning_rate='constant',penalty='l2',alpha=0.001)
reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)
print(reg.coef_)
print(reg.intercept_)
print(r2_score(y_test,y_pred))

[  31.71927119 -113.34546155  393.90851319  266.22782251   -9.16341664
  -65.71901674 -167.71173451  105.14760294  358.03512292  103.05626994]
[137.33313083]
0.4040329417828833


In [41]:
from sklearn.datasets import load_diabetes
from sklearn.metrics import r2_score
import numpy as np
X,y = load_diabetes(return_X_y=True)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=4)
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(penalty='l2',max_iter=500,eta0=0.1,learning_rate='constant',alpha=0.001)
reg.fit(X_train,y_train)

y_pred = reg.predict(X_test)
print("R2 score",r2_score(y_test,y_pred))
print(reg.coef_)
print(reg.intercept_)

R2 score 0.4105100362841061
[  47.06730468 -140.09378282  356.66704044  260.80177801   -1.55527886
  -53.09596388 -166.4370528   137.19884027  319.70305651  104.1610178 ]
[140.10030012]


In [42]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=0.001, max_iter=500,solver='sparse_cg')
reg.fit(X_train,y_train)

y_pred = reg.predict(X_test)
print("R2 score",r2_score(y_test,y_pred))
print(reg.coef_)
print(reg.intercept_)

R2 score 0.46250101619914563
[  34.52192544 -290.84084076  482.40181344  368.0678662  -852.44873179
  501.59160336  180.11115788  270.76333979  759.73534372   37.4913546 ]
151.10198517439466


In [43]:
class MYGDR :
  def __init__(self,epochs,learning_rate,alpha):
    self.epochs = epochs
    self.learning_rate = learning_rate
    self.alpha = alpha
    self.coef_ = None
    self.intercept_ = None
  def fit(self,X_train,y_train):
      self.coef_ = np.ones(X_train.shape[1])
      self.intercept_ = 0
      thetha = np.insert(self.coef_,0,self.intercept_)

      X_train = np.insert(X_train,0,1,axis=1)

      for i in range(self.epochs):
          thetha_der = np.dot(X_train.T,X_train).dot(thetha) - np.dot(X_train.T,y_train) + self.alpha*thetha
          thetha = thetha - self.learning_rate*thetha_der

      self.coef_ = thetha[1:]
      self.intercept_ = thetha[0]

  def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_


In [44]:
reg = MYGDR(epochs=500,alpha=0.001,learning_rate=0.005)
reg.fit(X_train,y_train)

y_pred = reg.predict(X_test)
print("R2 score",r2_score(y_test,y_pred))
print(reg.coef_)
print(reg.intercept_)

R2 score 0.4738018280260913
[  46.65050914 -221.3750037   452.12080647  325.54248128  -29.09464178
  -96.47517735 -190.90017011  146.32900372  400.80267299   95.09048094]
150.86975316713472


In [45]:
class MYSGDR:
    def __init__(self, epochs, learning_rate, alpha):
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.alpha = alpha
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X_train, y_train):
        n_samples, n_features = X_train.shape
        self.coef_ = np.ones(n_features)
        self.intercept_ = 0

        theta = np.insert(self.coef_, 0, self.intercept_)

        X_train = np.insert(X_train, 0, 1, axis=1)

        for epoch in range(self.epochs):
            for i in range(n_samples):
                xi = X_train[i].reshape(1, -1)
                yi = y_train[i]
                prediction = np.dot(xi, theta)[0]
                error = prediction - yi

                grad = error * xi + self.alpha * theta
                theta = theta - self.learning_rate * grad.flatten()

        self.intercept_ = theta[0]
        self.coef_ = theta[1:]

    def predict(self, X_test):
        return np.dot(X_test, self.coef_) + self.intercept_


In [46]:
class MYMBGDR:
    def __init__(self, epochs, learning_rate, alpha, batch_size):
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.alpha = alpha
        self.batch_size = batch_size
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X_train, y_train):
        n_samples, n_features = X_train.shape
        self.coef_ = np.ones(n_features)
        self.intercept_ = 0

        theta = np.insert(self.coef_, 0, self.intercept_)

        X_train = np.insert(X_train, 0, 1, axis=1)

        for epoch in range(self.epochs):
            indices = np.random.permutation(n_samples)
            X_shuffled = X_train[indices]
            y_shuffled = y_train[indices]

            for i in range(0, n_samples, self.batch_size):
                X_batch = X_shuffled[i:i + self.batch_size]
                y_batch = y_shuffled[i:i + self.batch_size]

                prediction = np.dot(X_batch, theta)
                error = prediction - y_batch

                grad = np.dot(X_batch.T, error) / len(X_batch) + self.alpha * theta
                theta = theta - self.learning_rate * grad

        self.intercept_ = theta[0]
        self.coef_ = theta[1:]

    def predict(self, X_test):
        return np.dot(X_test, self.coef_) + self.intercept_



In [47]:

# Train and evaluate SGD
sgd_model = MYSGDR(epochs=100, learning_rate=0.01, alpha=0.001)
sgd_model.fit(X_train, y_train)
y_pred_sgd = sgd_model.predict(X_test)
print("R2 score for Stochastic Gradient Descent:", r2_score(y_test, y_pred_sgd))

# Train and evaluate Mini-Batch GD
mbgd_model = MYMBGDR(epochs=100, learning_rate=0.01, alpha=0.001, batch_size=16)
mbgd_model.fit(X_train, y_train)
y_pred_mbgd = mbgd_model.predict(X_test)
print("R2 score for Mini-Batch Gradient Descent:", r2_score(y_test, y_pred_mbgd))


R2 score for Stochastic Gradient Descent: 0.4268156798268733
R2 score for Mini-Batch Gradient Descent: 0.12499860239656002
